##### 导入库

In [7]:
import pandas as pd
import numpy as np
import math

##### 选择数据

In [8]:
n='★阴性整合.csv'
p='★阳性整合.csv'
p='纯冰毒-重复.csv'
# p='纯海洛因-重复.csv'
# n='110正常人.csv'
# # n='0913-林职-阴性-55人.csv'
# p='纯冰毒.csv'
# n='1阴-66单条数据.csv'
# p='纯海洛因.csv'
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

##### 读取数据

In [9]:
path='E:\\desktop\\data\\'
n_fea = pd.read_csv(path+'fea\\[特征]'+n,encoding="gbk")
n_fea.drop(columns=['name'],axis = 1,inplace=True)

p_fea = pd.read_csv(path+'fea\\[特征]'+p,encoding="gbk")
p_fea.drop(columns=['name'],axis = 1,inplace=True)

fea_list=list(n_fea.columns)

np_fea=pd.concat([n_fea,p_fea],axis=0)
np_fea=np_fea[fea_list]
target=[0 for i in range(len(n_fea))]+[1 for i in range(len(p_fea))]
np_fea['target']=target

In [10]:
def iv_count(data, var, target ):#, weight
    ''' 计算iv值
    Args:
        data: DataFrame，拟操作的数据集
        var: String，拟计算IV值的变量名称
        target: String，Y列名称
    Returns:
        IV值， float
    '''
    value_list = set(list(np.unique(data[var])))
    iv = 0
    data_bad = pd.Series(data[data[target]==1][var].values, index=data[data[target]==1].index)
    data_good = pd.Series(data[data[target]==0][var].values, index=data[data[target]==0].index)
    len_bad = len(data_bad)
    len_good = len(data_good)
    for value in value_list:
        # 判断是否某类是否为0，避免出现无穷小值和无穷大值
        if sum(data_bad == value) == 0:
            bad_rate = 1 / len_bad
        else:
            bad_rate = sum(data_bad == value) / len_bad
        if sum(data_good == value) == 0:
            good_rate = 1 / len_good
        else:
            good_rate = sum(data_good == value) / len_good
        iv += (good_rate - bad_rate) * math.log(good_rate / bad_rate,2)
        # print(value,iv)
    return iv

In [11]:
def select_rf(df,target,imp_num=None):
    """
    df:数据集
    target:目标变量的字段名
    imp_num:筛选变量的个数
    
    return:
    rf_fea_imp:变量的特征重要性
    rf_select_col:筛选出的变量
    """
    x = df.drop([target],axis=1)
    y = df[target]
    rfmodel = RandomForestClassifier(random_state=0)
    rfmodel = rfmodel.fit(x,y)
    rf_fea_imp = pd.DataFrame({'col':list(x.columns),
                               'imp':rfmodel.feature_importances_})
    rf_fea_imp = rf_fea_imp.sort_values('imp',ascending=False).reset_index(drop=True).iloc[:imp_num,:]
    rf_select_col = list(rf_fea_imp.col)
    return rf_fea_imp,rf_select_col


# xgboost筛选变量
def select_xgboost(df,target,imp_num=None):
    """
    df:数据集
    target:目标变量的字段名
    imp_num:筛选变量的个数
    
    return:
    xg_fea_imp:变量的特征重要性
    xg_select_col:筛选出的变量
    """
    x = df.drop([target],axis=1)
    y = df[target]
    xgmodel = XGBClassifier(random_state=0)
    xgmodel = xgmodel.fit(x,y,eval_metric='auc')
    xg_fea_imp = pd.DataFrame({'col':list(x.columns),
                               'imp':xgmodel.feature_importances_})
    xg_fea_imp = xg_fea_imp.sort_values('imp',ascending=False).reset_index(drop=True).iloc[:imp_num,:]
    xg_select_col = list(xg_fea_imp.col)
    print(xg_fea_imp)
    return xg_fea_imp,xg_select_col

In [12]:
#差值辨别比
gap_dict={}
iv_dict={}
for fea in fea_list:
    n_col=list(n_fea[fea])
    p_col=list(p_fea[fea])

    gap_list=[]
    gap_up=0
    gap_down=0
    for a in range(len(n_col)):
        gaps=[]
        for b in range(len(p_col)):
            gap=p_col[b]-n_col[a]
            gaps.append(gap)
            if gap>0:
                gap_up=gap_up+1
            elif gap<0:
                gap_down=gap_down+1
        gap_list.append(gaps)

    gap_bili=0
    if (gap_up+gap_down)!=0:
        if gap_up/(gap_up+gap_down)>0.5:
            gap_bili=gap_up/(gap_up+gap_down)
        else:
            gap_bili=gap_down/(gap_up+gap_down)

    gap_dict[fea]=round(gap_bili*100,2)
    
    iv_dict[fea]=iv_count(np_fea, fea, 'target')

    
gap_dict = dict(sorted(gap_dict.items(), key=lambda x: x[1],reverse=True))
iv_dict = dict(sorted(iv_dict.items(), key=lambda x: x[1],reverse=True))
iv_dict

# print(select_rf(np_fea,'target'))
select_xgboost(np_fea,'target')

                                          col       imp
0                                   stability  0.130232
1                         attr_"abs"__coeff_1  0.081405
2                        attr_"imag"__coeff_3  0.055091
3    attr_"stderr"__chunk_len_10__f_agg_"max"  0.050089
4    attr_"stderr"__chunk_len_50__f_agg_"min"  0.049165
5                                          F1  0.046640
6                                    maxvalue  0.044028
7                                       time2  0.037642
8                       attr_"angle"__coeff_3  0.035187
9                       attr_"angle"__coeff_6  0.033318
10                       attr_"imag"__coeff_6  0.031467
11                                   minvalue  0.027835
12    attr_"stderr"__chunk_len_5__f_agg_"max"  0.027415
13                                 std1st_der  0.025193
14                                    Rrange1  0.024644
15                                      sdigu  0.023821
16                                    Rrange2  0

d:\anaconada\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


(                                          col       imp
 0                                   stability  0.130232
 1                         attr_"abs"__coeff_1  0.081405
 2                        attr_"imag"__coeff_3  0.055091
 3    attr_"stderr"__chunk_len_10__f_agg_"max"  0.050089
 4    attr_"stderr"__chunk_len_50__f_agg_"min"  0.049165
 5                                          F1  0.046640
 6                                    maxvalue  0.044028
 7                                       time2  0.037642
 8                       attr_"angle"__coeff_3  0.035187
 9                       attr_"angle"__coeff_6  0.033318
 10                       attr_"imag"__coeff_6  0.031467
 11                                   minvalue  0.027835
 12    attr_"stderr"__chunk_len_5__f_agg_"max"  0.027415
 13                                 std1st_der  0.025193
 14                                    Rrange1  0.024644
 15                                      sdigu  0.023821
 16                            